In [2]:
from fastapi import FastAPI, HTTPException, Header
from pydantic import BaseModel, RootModel, Field
from typing import List, Optional, Dict, Any
import httpx
import json

In [5]:
# Input model for the request payload
class Location(BaseModel):
    location_id: Optional[int] = Field(None, alias="location-id")
    location_name: Optional[str] = Field(None, alias="location-name")
    address: Optional[str] = None
    city: Optional[str] = None
    state: Optional[str] = None
    country: Optional[str] = None
    clli_code: Optional[str] = Field(None, alias="clli-code")

class OutageInfo(BaseModel):
    number_of_outages: Optional[int] = Field(None, alias="number-of-outages")
    outage_duration: Optional[int] = Field(None, alias="outage-duration")
    outage_unit_of_measure: Optional[str] = Field(None, alias="outage-unit-of-measure")
    outages: Optional[List[dict]] = None  # Handling the outages field if it's provided

class OutageDetails(BaseModel):
    outage_info: Optional[OutageInfo] = Field(None, alias="outage-info")

class MOP(BaseModel):
    mop_url: Optional[str] = Field(None, alias="mop-url")
    mop_comments: Optional[str] = Field(None, alias="mop-comments")
    backout_duration: Optional[str] = Field(None, alias="backout-duration")
    mop_id: Optional[str] = Field(None, alias="mop-id")  # Optional field
    mop_files: Optional[List[dict]] = None  # Handling the mop-files field
    mop_steps: Optional[List[dict]] = None  # Handling the mop-steps field
    backout_procedure: Optional[List[dict]] = None  # Handling backout-procedure field

class NetworkElement(BaseModel):
    ne_type: Optional[str] = Field(None, alias="ne-type")
    ne_id: Optional[str] = Field(None, alias="ne-id")
    ne_options: Optional[dict] = None  # Handling the ne-options field
    ne_location: Optional[dict] = None  # Handling the ne-location field
    ne_asp: Optional[List[dict]] = None  # Handling ne-asp field
    ne_application: Optional[dict] = None  # Handling ne-application field
    ne_site: Optional[dict] = None  # Handling ne-site field
    ne_virtual_element: Optional[List[dict]] = None  # Handling ne-virtual-element field

class ChangeRequestPayload(BaseModel):
    reference_id: Optional[str] = Field(None, alias="reference-id")
    service_impact: Optional[str] = Field(None, alias="service-impact")
    risk_level: Optional[str] = Field(None, alias="risk-level")
    requester: Optional[str] = None
    activity_category: Optional[str] = Field(None, alias="activity-category")
    activity_type: Optional[str] = Field(None, alias="activity-type")
    ticket_number: Optional[str] = Field(None, alias="ticket-number")
    description: Optional[str] = None
    network: Optional[str] = None
    subnetwork: Optional[str] = None
    location: Optional[Location] = None
    scheduled_start_date_time: Optional[str] = Field(None, alias="scheduled-start-date-time")
    scheduled_end_date_time: Optional[str] = Field(None, alias="scheduled-end-date-time")
    outage_details: Optional[OutageDetails] = Field(None, alias="outage-details")
    mop: Optional[List[MOP]] = None
    submitter: Optional[str] = None
    network_elements: Optional[List[NetworkElement]] = Field(None, alias="network-elements")
    individual_cell_site: Optional[bool] = Field(None, alias="individual-cell-site")  # Optional field
    ticket_source: Optional[str] = Field(None, alias="ticket-source")  # Optional field
    mpe_id: Optional[int] = Field(None, alias="mpe-id")  # Optional field
    break_fix: Optional[bool] = Field(None, alias="break-fix")  # Optional field
    contacts: Optional[List[dict]] = None  # Optional contacts field

In [19]:
class OccLocation(BaseModel):
    city: Optional[str] = None
    state: Optional[str] = None
    country: Optional[str] = None
    street: Optional[str] = None

class Location(BaseModel):
    vz_loc_id: Optional[str] = Field(None, alias="vz-loc-id")
    vz_loc_name: Optional[str] = Field(None, alias="vz-loc-name")
    source_system: Optional[str] = Field(None, alias="source-system")
    source_system_site_id: Optional[str] = Field(None, alias="source-system-site-id")
    clli: Optional[str] = None
    sensitive_clli: Optional[bool] = Field(None, alias="sensitive-clli")
    id: Optional[int] = None
    name: Optional[str] = None
    address: Optional[str] = None
    city: Optional[str] = None
    state: Optional[str] = None
    country: Optional[str] = None
    site_codes: Optional[List[dict]] = Field(None, alias="site-codes")
    timezone: Optional[str] = None
    timezone_id: Optional[str] = Field(None, alias="timezone-id")
    market_id: Optional[int] = Field(None, alias="market-id")
    market_name: Optional[str] = Field(None, alias="market-name")
    submarket_id: Optional[int] = Field(None, alias="submarket-id")
    submarket_name: Optional[str] = Field(None, alias="submarket-name")

class User(BaseModel):
    user_id: Optional[str] = Field(None, alias="user-id")
    display_name: Optional[str] = Field(None, alias="display-name")
    email: Optional[str] = None

class RelatedTicket(BaseModel):
    ticket_source_id: Optional[int] = Field(None, alias="ticket-source-id")
    ticket_source_name: Optional[str] = Field(None, alias="ticket-source-name")
    ticket_number: Optional[str] = Field(None, alias="ticket-number")

class MopFile(BaseModel):
    file_name: Optional[str] = Field(None, alias="file-name")
    file_path: Optional[str] = Field(None, alias="file-path")

class MopStep(BaseModel):
    step_seq: Optional[int] = Field(None, alias="step-seq")
    step_details: Optional[str] = Field(None, alias="step-details")

class RequestNE(BaseModel):
    item_id: Optional[int] = Field(None, alias="item-id")
    item_status_id: Optional[int] = Field(None, alias="item-status-id")
    item_status_name: Optional[str] = Field(None, alias="item-status-name")
    implementation_status_id: Optional[int] = Field(None, alias="implementation-status-id")
    implementation_status_name: Optional[str] = Field(None, alias="implementation-status-name")
    actual_start_date_time: Optional[str] = Field(None, alias="actual-start-date-time")
    actual_end_date_time: Optional[str] = Field(None, alias="actual-end-date-time")
    ne_id: Optional[str] = Field(None, alias="ne-id")
    ne_type: Optional[str] = Field(None, alias="ne-type")
    ne_asp: Optional[List[dict]] = None  # Placeholder for nested 'ne-asp' list
    ne_application: Optional[dict] = None  # Placeholder for 'ne-application' dictionary
    ne_site: Optional[dict] = None  # Placeholder for 'ne-site' dictionary
    ne_virtual_element: Optional[List[dict]] = None  # Placeholder for 'ne-virtual-element' list
    number_of_outages: Optional[int] = Field(None, alias="number-of-outages")
    outage_duration: Optional[int] = Field(None, alias="outage-duration")
    outage_unit_of_measure: Optional[str] = Field(None, alias="outage-unit-of-measure")
    outages: Optional[List[dict]] = None  # Placeholder for 'outages' list

class ChangeRequestResponseItem(BaseModel):
    request_id: Optional[int] = Field(None, alias="request-id")
    plan_id: Optional[int] = Field(None, alias="plan-id")
    request_status: Optional[str] = Field(None, alias="request-status")
    approval_comments: Optional[str] = Field(None, alias="approval-comments")
    scheduled_start_date_time: Optional[str] = Field(None, alias="scheduled-start-date-time")
    scheduled_end_date_time: Optional[str] = Field(None, alias="scheduled-end-date-time")
    network_id: Optional[int] = Field(None, alias="network-id")
    network_name: Optional[str] = Field(None, alias="network-name")
    lead_time_violated: Optional[bool] = Field(None, alias="lead-time-violated")
    maintenance_window_violated: Optional[bool] = Field(None, alias="maintenance-window-violated")
    subnetwork_id: Optional[int] = Field(None, alias="subnetwork-id")
    subnetwork_name: Optional[str] = Field(None, alias="subnetwork-name")
    individual_cell_site: Optional[bool] = Field(None, alias="individual-cell-site")
    mpe_id: Optional[int] = Field(None, alias="mpe-id")
    change_type_id: Optional[int] = Field(None, alias="change-type-id")
    change_type_name: Optional[str] = Field(None, alias="change-type-name")
    occ_location: Optional[OccLocation] = Field(None, alias="occ-location")
    carrier_name: Optional[str] = None
    impact_level_id: Optional[int] = Field(None, alias="impact-level-id")
    impact_level_name: Optional[str] = Field(None, alias="impact-level-name")
    service_impact_id: Optional[int] = Field(None, alias="service-impact-id")
    service_impact_name: Optional[str] = Field(None, alias="service-impact-name")
    risk_level_id: Optional[int] = Field(None, alias="risk-level-id")
    risk_level_name: Optional[str] = Field(None, alias="risk-level-name")
    activity_category_id: Optional[int] = Field(None, alias="activity-category-id")
    activity_category_name: Optional[str] = Field(None, alias="activity-category-name")
    activity_type_id: Optional[int] = Field(None, alias="activity-type-id")
    activity_type_name: Optional[str] = Field(None, alias="activity-type-name")
    description: Optional[str] = None
    location: Optional[Location] = None
    requester: Optional[User] = None
    submitter: Optional[User] = None
    implementer: Optional[User] = None
    related_ticket: Optional[RelatedTicket] = None
    reference_id: Optional[str] = Field(None, alias="reference-id")
    project_id: Optional[int] = Field(None, alias="project-id")
    created_date_time: Optional[str] = Field(None, alias="created-date-time")
    last_modified_by: Optional[User] = Field(None, alias="last-modified-by")
    last_modified_date_time: Optional[str] = Field(None, alias="last-modified-date-time")
    mop_files: Optional[List[MopFile]] = Field(None, alias="mop-files")
    mop_url: Optional[str] = Field(None, alias="mop-url")
    mop_id: Optional[str] = Field(None, alias="mop-id")
    mop_steps: Optional[List[MopStep]] = Field(None, alias="mop-steps")
    mop_comments: Optional[str] = Field(None, alias="mop-comments")
    backout_duration: Optional[str] = Field(None, alias="backout-duration")
    backout_procedure: Optional[List[List[MopStep]]] = Field(None, alias="backout-procedure")
    request_ne_list: Optional[List[RequestNE]] = Field(None, alias="request-ne-list")
    request_contacts: Optional[List[dict]] = Field(None, alias="request-contacts")
    request_conflicts: Optional[List[dict]] = Field(None, alias="request-conflicts")
    additionalProp1: Optional[Dict] = None


class ChangeRequestResponse(RootModel):
    root: List[List[ChangeRequestResponseItem]]

In [12]:
payload_str = """{
	"service-impact": "Non-Service Affecting",
	"risk-level" :"INCIDENT",
	"requester" :"SVC-gxov_kirke_np",
	"activity-category": "APPLICATION",
	"activity-type": "RESTART",
	"ticket-number" : "INC1001011",
	"description": "Automated Kirke plan/request creation by iEN-AP",
	"network": "CLOUD",
	"subnetwork": "CORE",
	"location": {
		"location-id": 48447605
	},
	"scheduled-start-date-time": "2024-12-26T23:00:00.000Z",
	"scheduled-end-date-time": "2024-12-26T23:05:00.000Z",
	"outage-details": {
		"outage-info": {
			"number-of-outages": 0,
			"outage-duration": 1,
			"outage-unit-of-measure": "Minutes"
		}
	},
	"mop": [
		{
			"mop-url": "https://www.verizon.com",
			"mop-comments": "MOP details not yet provided - contact requester.",
			"backout-duration": "00:01"
		}
	],
	"submitter": "SVC-gxov_kirke_np",
	"network-elements": [
	  {
		"ne-type": "Device",
		"ne-id": "twbgohaavzwcbsf-y-or-e1-000"
	 }
	 ]
}"""

In [13]:
data = json.loads(payload_str)  # Convert JSON string to a dictionary
data

{'service-impact': 'Non-Service Affecting',
 'risk-level': 'INCIDENT',
 'requester': 'SVC-gxov_kirke_np',
 'activity-category': 'APPLICATION',
 'activity-type': 'RESTART',
 'ticket-number': 'INC1001011',
 'description': 'Automated Kirke plan/request creation by iEN-AP',
 'network': 'CLOUD',
 'subnetwork': 'CORE',
 'location': {'location-id': 48447605},
 'scheduled-start-date-time': '2024-12-26T23:00:00.000Z',
 'scheduled-end-date-time': '2024-12-26T23:05:00.000Z',
 'outage-details': {'outage-info': {'number-of-outages': 0,
   'outage-duration': 1,
   'outage-unit-of-measure': 'Minutes'}},
 'mop': [{'mop-url': 'https://www.verizon.com',
   'mop-comments': 'MOP details not yet provided - contact requester.',
   'backout-duration': '00:01'}],
 'submitter': 'SVC-gxov_kirke_np',
 'network-elements': [{'ne-type': 'Device',
   'ne-id': 'twbgohaavzwcbsf-y-or-e1-000'}]}

In [14]:
payload = ChangeRequestPayload(**data)  # Validate and create the Pydantic model instance
payload

ChangeRequestPayload(reference_id=None, service_impact='Non-Service Affecting', risk_level='INCIDENT', requester='SVC-gxov_kirke_np', activity_category='APPLICATION', activity_type='RESTART', ticket_number='INC1001011', description='Automated Kirke plan/request creation by iEN-AP', network='CLOUD', subnetwork='CORE', location=Location(location_id=48447605, location_name=None, address=None, city=None, state=None, country=None, clli_code=None), scheduled_start_date_time='2024-12-26T23:00:00.000Z', scheduled_end_date_time='2024-12-26T23:05:00.000Z', outage_details=OutageDetails(outage_info=OutageInfo(number_of_outages=0, outage_duration=1, outage_unit_of_measure='Minutes', outages=None)), mop=[MOP(mop_url='https://www.verizon.com', mop_comments='MOP details not yet provided - contact requester.', backout_duration='00:01', mop_id=None, mop_files=None, mop_steps=None, backout_procedure=None)], submitter='SVC-gxov_kirke_np', network_elements=[NetworkElement(ne_type='Device', ne_id='twbgohaav

In [23]:
CHANGE_REQUEST_URL = "https://oa-uat.ebiz.verizon.com/msjv-kirke-changemanagement/changerequests/v2"

headers = {
    "Content-Type": "application/json",
    "accept": "application/json",
    "Authorization": "authorization",
    "jwtToken": "jwt_token",
}
headers

{'Content-Type': 'application/json',
 'accept': 'application/json',
 'Authorization': 'authorization',
 'jwtToken': 'jwt_token'}

In [25]:
payload.model_dump(by_alias=True)

{'reference-id': None,
 'service-impact': 'Non-Service Affecting',
 'risk-level': 'INCIDENT',
 'requester': 'SVC-gxov_kirke_np',
 'activity-category': 'APPLICATION',
 'activity-type': 'RESTART',
 'ticket-number': 'INC1001011',
 'description': 'Automated Kirke plan/request creation by iEN-AP',
 'network': 'CLOUD',
 'subnetwork': 'CORE',
 'location': {'location-id': 48447605,
  'location-name': None,
  'address': None,
  'city': None,
  'state': None,
  'country': None,
  'clli-code': None},
 'scheduled-start-date-time': '2024-12-26T23:00:00.000Z',
 'scheduled-end-date-time': '2024-12-26T23:05:00.000Z',
 'outage-details': {'outage-info': {'number-of-outages': 0,
   'outage-duration': 1,
   'outage-unit-of-measure': 'Minutes',
   'outages': None}},
 'mop': [{'mop-url': 'https://www.verizon.com',
   'mop-comments': 'MOP details not yet provided - contact requester.',
   'backout-duration': '00:01',
   'mop-id': None,
   'mop_files': None,
   'mop_steps': None,
   'backout_procedure': None}

In [17]:
async with httpx.AsyncClient() as client:
    response = await client.post(
        CHANGE_REQUEST_URL, headers=headers, json=payload.model_dump(by_alias=True)
    )

response

ConnectError: [Errno 8] nodename nor servname provided, or not known

In [18]:
response.status_code

NameError: name 'response' is not defined

In [ ]:
response_data = response.json()
response_data

In [20]:
ChangeRequestResponse.model_validate(response_data)

NameError: name 'response_data' is not defined